<a href="https://colab.research.google.com/github/prathwishshetty/DS5500_HW_3/blob/master/DS5500_HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1>DS 5500 Homework 3 - Due Nov. 11</h1></center>

##Instructions  
<p>  
Create a new public Github repository for this homework assignment. The repository should include all of
the code necessary to reproduce your submitted solutions. Do not include the raw data in the repository.
</p>

*Use the README.md of the Github repository to present your solutions. Answer all questions completely for
full credit, including figures and tables where appropriate. For each problem, either provide relevant inline
code snippets, or cite the source file where the relevant code lives (with line numbers if appropriate).*

Describe any data processing steps (transformation, filtering, etc.) you perform when solving each problem,
providing reasoning where appropriate. You may need to be creative when deciding how to approach each
problem, as there may not be a single “correct” solution.

Your solutions should be posted as a **public** *note* on Piazza in the hw3 folder with the title “[hw3] - your name name”. Include in the body of the note a link to the Github repository with your solutions.

##Overview

In this homework assignment, you are a data scientist working for the U.S. federal government. Due to
budget cuts for education, your office has been tasked with cutting federal funding to some number of school
districts. Your supervisor has asked you to develop a recommendation and objective justification for their
decision using data analysis.

Due to related budget cuts, the most recent fiscal data you have to work with is from 2015-16. Download the
2015-16 district-level fiscal data from the National Center for Education Statistics’ Common Core of Data:

*   https://nces.ed.gov/ccd/f33agency.asp

For helping you make your decision, it may be helpful to have some performance metrics for each district.
You can download the 2015-16 data for distrct-level statistics on graduation rate and state assessments on
mathematics and reading/language arts from the EDFacts website:
* https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html  

These datasets can be linked based on the LEA IDs

### Problem 1

Import and explore the district-level fiscal data from 2015-16.

Rank and visualize the states that take in the most federal funding (revenue).

Which states spend the most federal funding per student?

In [0]:
#@title Download data

!wget https://nces.ed.gov/ccd/data/zip/sdf16_1a_sas7bdat.zip
!unzip sdf16_1a_sas7bdat.zip
!wget https://www2.ed.gov/about/inits/ed/edfacts/data-files/math-achievement-sch-sy2015-16.csv
!wget https://www2.ed.gov/about/inits/ed/edfacts/data-files/rla-achievement-sch-sy2015-16.csv

--2019-11-11 17:02:51--  https://nces.ed.gov/ccd/data/zip/sdf16_1a_sas7bdat.zip
Resolving nces.ed.gov (nces.ed.gov)... 63.145.228.23, 2001:428:7003:11::23
Connecting to nces.ed.gov (nces.ed.gov)|63.145.228.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4646961 (4.4M) [application/x-zip-compressed]
Saving to: ‘sdf16_1a_sas7bdat.zip’

sdf16_1a_sas7bdat.z 100%[===================>]   4.43M   392KB/s    in 12s     

2019-11-11 17:03:04 (371 KB/s) - ‘sdf16_1a_sas7bdat.zip’ saved [4646961/4646961]

Archive:  sdf16_1a_sas7bdat.zip
  inflating: sdf16_1a.sas7bdat       
--2019-11-11 17:03:07--  https://www2.ed.gov/about/inits/ed/edfacts/data-files/math-achievement-sch-sy2015-16.csv
Resolving www2.ed.gov (www2.ed.gov)... 165.224.131.182
Connecting to www2.ed.gov (www2.ed.gov)|165.224.131.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48805549 (47M) [text/csv]
Saving to: ‘math-achievement-sch-sy2015-16.csv’

math-achievement-sc 100%[===

In [0]:
 import pandas as pd
import numpy as np

data = pd.read_sas('sdf16_1a.sas7bdat', format = 'sas7bdat', encoding="utf-8")
math = pd.read_csv('math-achievement-sch-sy2015-16.csv',error_bad_lines=False, index_col=False, dtype='unicode')
rla  = pd.read_csv('rla-achievement-sch-sy2015-16.csv',error_bad_lines=False, index_col=False, dtype='unicode')

In [5]:
data_state_revenue = data.groupby(['STNAME'])['TFEDREV'].sum().reset_index()
import plotly.express as px
fig = px.bar(data_state_revenue, x='STNAME', y='TFEDREV')
fig.show()

In [6]:
pd.set_option('display.float_format', '{:,.0f}'.format)
data_state_revenue.sort_values(by=['TFEDREV'],ascending=False).head(10)

,STNAME,TFEDREV
4,California,"7,709,274,795"
43,Texas,"6,194,316,950"
32,New York,"3,374,793,578"
9,Florida,"3,147,328,988"
13,Illinois,"2,334,944,853"
38,Pennsylvania,"2,037,314,939"
35,Ohio,"1,837,962,930"
10,Georgia,"1,815,241,986"
22,Michigan,"1,731,033,937"
33,North Carolina,"1,587,975,932"


In [0]:
data_grouped=  data.groupby(['STNAME'])['TOTALEXP','V33'].sum().reset_index()
data_grouped['exp_per_stud'] = data_grouped['TOTALEXP']/data_grouped['V33']

In [8]:
import plotly.express as px
fig = px.bar(data_grouped, x='STNAME', y='exp_per_stud')
fig.show()

In [9]:
data_grouped.sort_values(by=['exp_per_stud'],ascending=False)[['STNAME','exp_per_stud']].head(10)

,STNAME,exp_per_stud
8,District of Columbia,"27,322"
32,New York,"25,235"
45,Vermont,"24,373"
30,New Jersey,"21,641"
50,Wyoming,"21,479"
6,Connecticut,"20,977"
1,Alaska,"19,800"
38,Pennsylvania,"19,216"
21,Massachusetts,"18,335"
34,North Dakota,"18,093"


### Problem 2

Visualize the relationship between school districts’ total revenue and expenditures.

Which states have the most debt per student?

In [10]:
pd.set_option('display.float_format', '{:,.10f}'.format)

data_rev_exp = data[['LEAID','STNAME','TOTALEXP', 'TOTALREV']].query('TOTALREV>0 and TOTALEXP>0').copy()
data_rev_exp['Rev_Exp'] =  data_rev_exp['TOTALEXP']/data_rev_exp['TOTALREV']
#removing outliers
data_rev_exp2 = data_rev_exp.query('Rev_Exp<10')
import plotly.express as px
fig = px.histogram(data_rev_exp2, x="Rev_Exp",range_x=[0,3],nbins=1000)
fig.show()

In [11]:
pd.set_option('display.float_format', '{:,.0f}'.format)
data_debt = data[['STNAME','V33','_31F','_66V']].copy()
data_debt['totalDebt'] = data_debt['_31F']+ data_debt['_66V']
data_debt_grouped=  data_debt.groupby(['STNAME'])['V33','totalDebt'].sum().reset_index()
data_debt_grouped['debt_per_stud'] = data_debt_grouped['totalDebt']/data_debt_grouped['V33']
data_debt_grouped.sort_values(by=['debt_per_stud'],ascending=False)[['STNAME','debt_per_stud']].head(10)

,STNAME,debt_per_stud
22,Michigan,"3,079"
43,Texas,"2,955"
38,Pennsylvania,"2,757"
32,New York,"2,213"
28,Nevada,"1,979"
3,Arkansas,"1,685"
1,Alaska,"1,613"
13,Illinois,"1,532"
14,Indiana,"1,460"
40,South Carolina,"1,408"


### Problem 3

The district-level performance metrics from EDFacts may be useful in your decision.

However, to protect student privacy, the data in these datasets has been heavily “blurred” to prevent students
from being identified. Therefore, most of the numeric metrics are presented as ranges in string format. In
addition, censored and missing data must be imputed.

Write and explain a function for processing a single column of “blurred” metrics into usable numeric values.



In [0]:
def columnFixer(data,column):

    data['temp_for_PS'] = data[column].fillna('PS')
    colVals = data.temp_for_PS.copy()

    def stringFixer(string):
        if string.isdigit()==True:x=int(string)
        elif "-" in string: x=int(np.mean([int(x) for x in string.split('-')]))
        elif string=='PS': x=np.nan
        elif string.isalnum()==True:
            alpha = re.findall("[a-zA-Z]+",string)[0]
            num   = int(re.findall(r'\d+', string)[0]) 
            if alpha == "LT":   x= int((0+(num-1))/2)
            elif alpha == "LE": x= int((0+num)/2)
            elif alpha == "GT": x= int(((num+1)+100)/2)
            elif alpha == "GE": x= int((num+100)/2)
        else:
            x = np.nan
        return x

    colVals_proc = [stringFixer(x) for x in colVals]

    col_name = column+'_proc'

    data[col_name]= colVals_proc
    data[col_name] =  data[col_name].fillna(np.nanmean(colVals_proc, axis=0))
    return data

The function `columnFixer` tries to fix the data in 2 steps:

* Unbundling: Here if the values are in a range them we assign the median of that range as the value for a particular value

* Imputing: For missing values we repalce the values by the mean of that column

Use it to process and then visualize the distribution of a performance metric of your choice.

In [118]:
import re
math_proc = columnFixer(math,'ALL_MTH00PCTPROF_1516')
fig = px.histogram(math_proc, x="ALL_MTH00PCTPROF_1516",range_x=[0,100],nbins=10)
fig.show()

###Problem 4

You are tasked with cutting 15% of the U.S. federal budget currently being spent on funding school districts.

How much money is this?


In [114]:
print('Amount of the federal budget that will be cut: ${:,.0f}'.format(sum(data[data.TFEDREV >0].TFEDREV)*0.15))

Amount of the federal budget that will be cut: $8,340,411,300


Choose which school districts will have their funding cut and how this will be done.

(You should produce a table of LEA IDs and the dollar amount by which their federal funding will be cut –
you do not need print the entire table.)

In [115]:
#@title

pd.options.mode.chained_assignment = None

math_sub = math[['LEAID','ALL_MTH00PCTPROF_1516']].copy()
math_sub = columnFixer(math_sub,'ALL_MTH00PCTPROF_1516')
math_sub_grouped = math_sub.groupby(['LEAID'])['ALL_MTH00PCTPROF_1516_proc'].mean().reset_index()
rla_sub  =  rla[['LEAID','ALL_RLA00PCTPROF_1516']].copy()
rla_sub = columnFixer(rla_sub,'ALL_RLA00PCTPROF_1516')
rla_sub_grouped = rla_sub.groupby(['LEAID'])['ALL_RLA00PCTPROF_1516_proc'].mean().reset_index()
data_math     = pd.merge(data,     math_sub_grouped, on='LEAID', how='left')
data_math_rla = pd.merge(data_math,rla_sub_grouped,  on='LEAID',how='left') 
data_math_rla = data_math_rla[data_math_rla['ALL_MTH00PCTPROF_1516_proc'].notnull()]
data_math_rla = data_math_rla[data_math_rla['ALL_RLA00PCTPROF_1516_proc'].notnull()]
data_debt_grouped = data_math_rla[['LEAID','NAME','STNAME','TFEDREV','ALL_MTH00PCTPROF_1516_proc','ALL_RLA00PCTPROF_1516_proc']]
data_debt_grouped['Avg_Score'] = (data_debt_grouped['ALL_MTH00PCTPROF_1516_proc'] + data_debt_grouped['ALL_RLA00PCTPROF_1516_proc'])/2
#funding cut 1
cut1 = data_debt_grouped.query('Avg_Score>75')
cut1['fed_cut'] = cut1['TFEDREV'].multiply(0.30)
cut1 = cut1[['LEAID','fed_cut']].copy()
cut2 = data_debt_grouped.query('Avg_Score>50 and Avg_Score<75')
cut2['fed_cut'] = cut2['TFEDREV'].multiply(0.25)
cut2 = cut2[['LEAID','fed_cut']].copy()
cut3 = data_debt_grouped.query('Avg_Score>25 and Avg_Score<50')
cut3['fed_cut'] = cut3['TFEDREV'].multiply(0.15)
cut3 = cut3[['LEAID','fed_cut']].copy()
main_cut = pd.concat([cut1,cut2,cut3])
main_cut.head(10)

,LEAID,fed_cut
140,0102490,"237,900"
174,0103430,"521,400"
266,0400069,"4,500"
268,0400071,0
306,0400126,0
312,0400134,"34,500"
313,0400135,"66,300"
329,0400165,"27,300"
350,0400211,0
365,0400232,0


In [117]:
print('Savings from the federal cut achieved: ${:,.0f}'.format(main_cut['fed_cut'].sum()))

Savings from the federal cut achieved: $8,673,157,483


###Problem 5
Provide a statement for your supervisor justifying your decisions on which school districts will lose funding.

I have used a multi level funding cut so as to not impact any school district too much, hence we start off with a 30% cut for school districts with Avergae Score across math and LEA of greater than 75%.

And in the same way to reduce burden on already worse performing schools, i only cut their funding by 25% if they score above 50% and below 75%.

The least funding cut was doen to schools with student scoring above 25% and below 50%. I cut their funding by 15%.

The schools with scores from 0 to 25% saw no funding cut.